# Test model structures

This notebook outlines how to use the hydrologic portion of the Potions model, including setting up a model, running, and some light calibration


In [1]:
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import potions as pt
import warnings
warnings.filterwarnings("ignore")

## Load data


In [2]:
data_path: str = "../input/Sleepers_Results.txt"
df = pd.read_csv(data_path, sep="\\s+", index_col="Date", parse_dates=True)
df.head()

,Qsim,Qobs,Precipitation,Temperature,AET,PET,Snow,Snowcover,SM,Recharge,SUZ,SLZ,Q0,Q1,Q2,Qsim_rain,Qsim_snow
Date,,,,,,,,,,,,,,,,,
2016-10-01,0.498270,0.138,0.0,8.7,1.29,2.27,0.0,0.0,92.7,0.0,0.0,6.3,0.0,0.0,0.498270,0.496,0.002
2016-10-02,0.461498,0.170,0.0,9.8,0.94,1.68,0.0,0.0,91.7,0.0,0.0,5.8,0.0,0.0,0.461498,0.460,0.002
2016-10-03,0.477553,0.207,4.3,11.7,1.08,1.91,0.0,0.0,94.3,0.7,0.0,6.0,0.0,0.0,0.477553,0.476,0.002
2016-10-04,0.442310,0.195,0.0,11.6,1.09,1.90,0.0,0.0,93.2,0.0,0.0,5.6,0.0,0.0,0.442310,0.441,0.002
2016-10-05,0.409667,0.163,0.0,12.1,1.30,2.30,0.0,0.0,91.9,0.0,0.0,5.1,0.0,0.0,0.409667,0.408,0.002


In [3]:
forc = pt.ForcingData(precip=df["Precipitation"], temp=df["Temperature"], pet=df["PET"])
meas_streamflow = df["Qobs"]

# Construct the model


In [4]:
mod = pt.HbvModel(lapse_rate=None)

In [5]:
res = mod.run(
    init_state=mod.default_init_state(),
    forc=forc,
    meas_streamflow=meas_streamflow,
)

In [15]:
(
    best_params,
    best_res,
    opt_res,
) = pt.HbvModel.simple_calibration(
    forc=forc, meas_streamflow=meas_streamflow, metric="nse", maxiter=10, polish=True
)
print(f"Best NSE: {round(best_res.nse, 2)}") # type: ignore
print(f"Best KGE: {round(best_res.kge, 2)}") # type: ignore

Best NSE: 0.87
Best KGE: 0.83


In [33]:
class ThreeLayerModel(pt.Model):
    structure = [
        [pt.SnowZone(name="snow")],
        [pt.SoilZone(name="soil")],
        [pt.GroundZoneB(name="ground")],
    ]

In [29]:
mod = ThreeLayerModel()

## A hillslope-riparian zone model


In [ ]:
class DualModel(pt.Model):
    structure = [
        [pt.SnowZone(name="snow_hs"), pt.SnowZone(name="snow_rp")],
        [pt.SoilZone(name="soil_hs"), pt.SoilZone(name="soil_rp")],
        [pt.GroundZoneB(name="ground_hs"), pt.GroundZoneB(name="ground_rp")],
    ]


In [ ]:
mod = DualModel(scales=[0.5, 0.5])
res = mod.run(forc=[forc, forc], meas_streamflow=meas_streamflow)

In [15]:
mod.to_array().shape

(19,)

In [16]:
mod2 = DualModel.from_array(mod.to_array())

In [17]:
dual_res = mod.simple_calibration(
    forc=[forc, forc],
    meas_streamflow=meas_streamflow,
    metric="nse",
    use_lapse_rates=False,
    num_threads=-1,
    polish=True
)

In [26]:
dual_res[1].nse

np.float64(0.8936374524883219)

## Now, try a mixed model

In [34]:
class CustomModel(pt.Model):
    structure = [
        [pt.SnowZone(name="snow_hs"), pt.SnowZone(name="snow_rp")],
        [pt.SoilZone(name="soil_hs"), pt.SoilZone(name="soil_rp")],
        [pt.GroundZone(name="shallow_hs"), pt.GroundZone(name="shallow_rp")],
        [pt.GroundZoneB(name="deep")],
    ]

In [23]:
custom_res = CustomModel.simple_calibration(
    forc=[forc, forc],
    meas_streamflow=meas_streamflow,
    metric="nse",
    use_lapse_rates=False,
    num_threads=-1,
    polish=True
)

## Triple model
In this model, we define a model with three zones

In [36]:
class TripleModel(pt.Model):
    structure = [
        [pt.SnowZone(name="snow_1"), pt.SnowZone(name="snow_2"), pt.SnowZone(name="snow_3")],
        [pt.SnowZone(name="soil_1"), pt.SoilZone(name="soil_2"), pt.SoilZone(name="soil_3")],
        [pt.GroundZoneB(name="ground_1"), pt.GroundZoneB(name="ground_2"), pt.GroundZoneB(name="ground_3")],
    ]

In [42]:
res = TripleModel.simple_calibration(
    forc=[forc] * 3,
    meas_streamflow=meas_streamflow,
    metric="nse",
    polish=True,
    maxiter=5
)

In [48]:
mod = TripleModel()

In [52]:
mod.run?

Signature:
mod.run(
    forc: 'ForcingData | list[ForcingData]',
    init_state: 'Optional[NDArray[f64]]' = None,
    streamflow: 'Optional[Series]' = None,
    average_elevation: 'Optional[float]' = None,
    elevations: 'Optional[list[float]]' = None,
    bounds: 'Optional[dict[str, tuple[float, float]]]' = None,
    verbose: 'bool' = False,
) -> 'HydroModelResults'
Docstring: Run the hydrologic simulation forward and calculate the objective functions
File:      ~/Documents/Research/Projects/potions/src/potions/model.py
Type:      method

In [56]:
%%prun 
res = mod.run(
    forc=[forc] * 3,
    streamflow=meas_streamflow
)

         433990 function calls (433787 primitive calls) in 0.195 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     6574    0.034    0.000    0.056    0.000 numeric.py:2337(isclose)
     3285    0.018    0.000    0.107    0.000 _zeros_py.py:109(newton)
     3285    0.011    0.000    0.129    0.000 _root_scalar.py:62(root_scalar)
      365    0.011    0.000    0.181    0.000 model.py:307(step)
     9859    0.009    0.000    0.017    0.000 hydro.py:97(mass_balance)
        3    0.008    0.003    0.010    0.003 base_events.py:1954(_run_once)
     3285    0.008    0.000    0.151    0.000 hydro.py:72(step)
     9855    0.006    0.000    0.006    0.000 {built-in method builtins.locals}
    32870    0.005    0.000    0.008    0.000 numeric.py:2429(<genexpr>)
     5841    0.005    0.000    0.014    0.000 {built-in method builtins.sum}
     9859    0.005    0.000    0.022    0.000 hydro.py:82(f)
        1    0.004    0.004    0.19